In [21]:
import ast
import gc
import os
import random
import re
import string
import warnings
from functools import partial
from itertools import islice, product
from typing import Tuple
from sklearn.feature_extraction.text import TfidfVectorizer
import artm
import nltk
import numpy as np
import optuna
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import wget
from corus import load_lenta
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from num2words import num2words
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm

warnings.filterwarnings("ignore")
pyLDAvis.enable_notebook()
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
!pip install gensim numpy==1.25.0 nltk bigartm num2words tqdm pymorphy2 optuna pyLDAvis wget corus

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 1.8 MB/s eta 0:00:00


In [7]:
RANDOM_SEED = 42

In [8]:
random.seed(RANDOM_SEED)
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [9]:
morph_analyzer = MorphAnalyzer(lang="ru")
tokenizer = WordPunctTokenizer()

In [10]:
russian_stopwords = stopwords.words("russian")
additional_sw = "мои оно мной мною мог могут мор мое мочь оба нам нами ними однако нему никуда наш нею неё наша наше наши очень отсюда вон вами ваш ваша ваше ваши весь всем всеми вся ими ею будем будете будешь буду будь будут кому кого которой которого которая которые который которых кем каждое каждая каждые каждый кажется та те тому собой тобой собою тобою тою хотеть хочешь свое свои твой своей своего своих твоя твоё сама сами теми само самом самому самой самого самим самими самих саму чему тебе такое такие также такая сих тех ту эта это этому туда этим этими этих абы аж ан благо буде вроде дабы едва ежели затем зато ибо итак кабы коли коль либо лишь нежели пока покамест покуда поскольку притом причем пускай пусть ровно сиречь словно также точно хотя чисто якобы "
pronouns = "я мы ты вы он она оно они себя мой твой ваш наш свой его ее их то это тот этот такой таков столько весь всякий сам самый каждый любой иной другой кто что какой каков чей сколько никто ничто некого нечего никакой ничей нисколько кто-то кое-кто кто-нибудь кто-либо что-то кое-что что-нибудь что-либо какой-то какой-либо какой-нибудь некто нечто некоторый некий"
conjunctions = "что чтобы как когда ибо пока будто словно если потому что оттого что так как так что лишь только как будто с тех пор как в связи с тем что для того чтобы кто как когда который какой где куда откуда"
digits = "ноль один два три четыре пять шесть семь восемь девять десять одиннадцать двенадцать тринадцать четырнадцать пятнадцать шестнадцать семнадцать восемнадцать девятнадцать двадцать тридцать сорок пятьдесят шестьдесят семьдесят восемьдесят девяносто сто"
modal_words = "вероятно возможно видимо по-видимому кажется наверное безусловно верно  действительно конечно несомненно разумеется"
particles = "да так точно ну да не ни неужели ли разве а что ли что за то-то как ну и ведь даже еще ведь уже все все-таки просто прямо вон это вот как словно будто точно как будто вроде как бы именно как раз подлинно ровно лишь только хоть всего исключительно вряд ли едва ли"
prepositions = "близ  вблизи  вдоль  вокруг  впереди  внутрь  внутри  возле  около  поверх  сверху  сверх  позади  сзади  сквозь  среди  прежде  мимо  вслед  согласно  подобно  навстречу  против  напротив  вопреки  после  кроме  вместе  вдали  наряду  совместно  согласно  нежели вроде от бишь до без аж тех раньше совсем только итак например из прямо ли следствие а поскольку благо пускай благодаря случае затем притом также связи время при чтоб просто того невзирая даром вместо точно покуда тогда зато ради ан буде прежде насчет раз причине тому так даже исходя коль кабы более ровно либо помимо как-то будто если словно лишь бы и не будь пор тоже разве чуть как хотя наряду потому пусть в равно между сверх ибо на судя то чтобы относительно или счет за но сравнению причем оттого есть когда уж ввиду тем для дабы чем хоть с вплоть скоро едва после той да вопреки ежели кроме сиречь же коли под абы несмотря все пока покамест паче прямо-таки перед что по вдруг якобы подобно"
evaluative = "наиболее наименее лучший больший высший низший худший более менее"

russian_stopwords.extend(additional_sw.split())
russian_stopwords.extend(pronouns.split())
russian_stopwords.extend(conjunctions.split())
russian_stopwords.extend(digits.split())
russian_stopwords.extend(modal_words.split())
russian_stopwords.extend(particles.split())
russian_stopwords.extend(prepositions.split())
russian_stopwords.extend(evaluative.split())
russian_stopwords = set(russian_stopwords)

## Скачиваем датасет, делаем предобработку

In [11]:
dataset_url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz"
wget.download(dataset_url)

'lenta-ru-news.csv.gz'

In [12]:
path = "lenta-ru-news.csv.gz"
records = load_lenta(path)

Датасет большой, поэтому возьмем 3000 записей

In [13]:
news = (record.title + ". " + record.text for record in islice(records, 3000))

### Делаем препроцессинг текста

In [14]:
def convert_digit_to_word(text):
    pattern = r"\d+"
    numbers = re.findall(pattern, text)
    for number in numbers:
        text = text.replace(number, num2words(number, lang="ru"))
    return text


def lemmatize(words) -> list[str]:
    return [morph_analyzer.parse(word)[0].normal_form for word in words]


def delete_stop_words(words):
    return [word for word in words if word not in russian_stopwords]


def delete_punctuation(text):
    return " ".join([word for word in tokenizer.tokenize(text) if word.isalnum()])


def tokenize_by_sentences(text: str) -> list[str]:
    return sent_tokenize(text)


def tokenize_by_words(text):
    return word_tokenize(text)


def convert_text_to_lowercase(text: str):
    return text.lower()


def preprocess(text: str) -> list[str]:
    sentences: list[str] = tokenize_by_sentences(text)
    preprocessed_sentences: list[str] = list()
    for sentence in sentences:
        sentence = convert_text_to_lowercase(sentence)
        sentence = convert_digit_to_word(sentence)
        sentence = delete_punctuation(sentence)
        words = tokenize_by_words(sentence)
        words = delete_stop_words(words)
        words = lemmatize(words)
        preprocessed_sentences.extend(words)
    return preprocessed_sentences

In [15]:
preprocessed_news = [" ".join(preprocess(record)) for record in tqdm(news)]

3000it [02:39, 18.75it/s]


In [16]:
preprocessed_news[0]

'назвать регион россия высокий смертность рак вица премьер социальный вопрос татьяна голиков рассказать какой регион россия зафиксировать высокий смертность рак сообщать риа новость слово голиков частый онкологический заболевание становиться причина смерть псковский тверской тульский орловский область севастополь вица премьер напомнить главный фактор смертность россия рак болезнь система кровообращение начало год стать известно смертность онкологический заболевание россиянин снизиться впервые год данные росстат два тысяча год рак умереть двести тысяча человек процент маленький год ранее'

## Обучение BigARTM

Подготовим словарь и корпус для LDA из датасета

In [20]:
tfidf_vec = TfidfVectorizer(max_features=1000, ngram_range=(1,2))
n_wd = np.array(tfidf_vec.fit_transform(preprocessed_news).todense()).T

In [22]:
bv = artm.BatchVectorizer(data_format='bow_n_wd', n_wd=n_wd, vocabulary=list(tfidf_vec.vocabulary_.keys()))

Начинаем эксперименты по подбору параметров. Будем подбирать num_topics, chunk_size, passes в разных сочетаниях и проверять качество на coherence модели

In [26]:
def train(
    num_topics: int,
    phi_tau,
    theta_tau,
    dictionary
):
    model = artm.ARTM(num_topics=num_topics, dictionary=dictionary, cache_theta=True)

    model.scores.add(artm.PerplexityScore(name='perplexity_score', dictionary=dictionary))

    model.regularizers.add(
        artm.SmoothSparsePhiRegularizer(
          name='SparsePhi',
          tau=phi_tau
      ),
    )
    model.regularizers.add(
        artm.SmoothSparseThetaRegularizer(
            name='SparseTheta',
            tau=theta_tau
        ),
    )
    model.fit_offline(bv, num_collection_passes=30)

    perplexity_score = model.score_tracker["perplexity_score"].last_value
    return perplexity_score, model

In [27]:
def objective(trial, dictionary):
    num_topics = trial.suggest_int("num_topics", 5, 50)
    phi_tau = trial.suggest_float("phi_tau", -1, 1)
    theta_tau = trial.suggest_float("theta_tau", -1, 1)
    perplexity, _ = train(
        num_topics=num_topics,
        phi_tau=phi_tau,
        theta_tau = theta_tau,
        dictionary=bv.dictionary,
    )
    return perplexity

In [28]:
%%time
study = optuna.create_study(direction="minimize")
study.optimize(
    partial(objective, dictionary=bv.dictionary),
    n_trials=25,
    show_progress_bar=True,
)

best_trial = study.best_trial
best_params = study.best_params

[I 2023-11-02 15:16:39,878] A new study created in memory with name: no-name-39f86446-5932-4509-a8a0-d77df1a94fba


  0%|          | 0/25 [00:00<?, ?it/s]

[I 2023-11-02 15:16:43,855] Trial 0 finished with value: 600.4246826171875 and parameters: {'num_topics': 6, 'phi_tau': 0.09341676121267528, 'theta_tau': 0.3672468206066801}. Best is trial 0 with value: 600.4246826171875.
[I 2023-11-02 15:16:48,274] Trial 1 finished with value: 834.675537109375 and parameters: {'num_topics': 30, 'phi_tau': 0.7631240175161351, 'theta_tau': -0.7186335946257774}. Best is trial 0 with value: 600.4246826171875.
[I 2023-11-02 15:16:54,928] Trial 2 finished with value: 515.775146484375 and parameters: {'num_topics': 18, 'phi_tau': -0.20563638473946488, 'theta_tau': 0.1648492797984693}. Best is trial 2 with value: 515.775146484375.
[I 2023-11-02 15:16:57,961] Trial 3 finished with value: 562.8359985351562 and parameters: {'num_topics': 6, 'phi_tau': 0.2251166670234117, 'theta_tau': 0.13928475594324508}. Best is trial 2 with value: 515.775146484375.
[I 2023-11-02 15:17:01,792] Trial 4 finished with value: 429.1389465332031 and parameters: {'num_topics': 15, 'ph

In [35]:
_, best_bigartm_model = train(**best_params, dictionary=bv.dictionary)

### Визуализируем результаты работы

In [38]:
def prepare_vis_data(model):
    try:
        phi = model.get_phi()
        theta = model.get_theta().to_numpy().T
        theta = theta / theta.sum(axis=1, keepdims=1)
        data = {'topic_term_dists': phi.to_numpy().T,
                'doc_topic_dists': theta,
                'doc_lengths': n_wd.sum(axis=0).tolist(),
                'vocab': phi.T.columns,
                'term_frequency': n_wd.sum(axis=1).tolist()}
        return data
    finally:
        if hasattr(model, 'dispose'):
            model.dispose()

In [39]:
model_data = prepare_vis_data(best_bigartm_model)
model_vis = pyLDAvis.prepare(**model_data)

In [40]:
model_vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
15     0.123882 -0.025952       1        1  9.455948
26     0.051905 -0.121863       2        1  6.755742
44    -0.040981 -0.247175       3        1  4.815989
45     0.037518  0.149967       4        1  4.626181
40     0.101778 -0.059721       5        1  3.893288
8      0.187194  0.098889       6        1  3.642441
9     -0.110855 -0.136050       7        1  3.318464
41     0.210773  0.002179       8        1  3.164236
16    -0.175659  0.047718       9        1  2.993443
13     0.116979 -0.081226      10        1  2.785959
31     0.114889 -0.166962      11        1  2.742270
23    -0.085379  0.020668      12        1  2.678445
22     0.095469  0.065193      13        1  2.497845
42     0.149825  0.056502      14        1  2.449244
34    -0.011221  0.048183      15        1  2.399580
3      0.057953  0.061347      16        1  2.285082
37     0.018597  0.140781      17        1  2.114201
0      0.093888  0.027622      18        1  2.079442
28    -0.026592  0.118281      19        1  2.053753
30     0.033991  0.067133      20        1  1.968056
11    -0.035546  0.062859      21        1  1.769004
4      0.007254  0.011570      22        1  1.746859
7      0.118108  0.026046      23        1  1.734555
43     0.082339 -0.139606      24        1  1.625233
29    -0.058509 -0.008680      25        1  1.531652
33     0.086700  0.043496      26        1  1.504144
5     -0.102393  0.131748      27        1  1.498415
27    -0.066011 -0.248941      28        1  1.468549
1      0.015364 -0.058127      29        1  1.454408
21     0.025995 -0.050089      30        1  1.429559
35    -0.083564  0.037565      31        1  1.317941
6     -0.096655  0.081938      32        1  1.312325
38     0.001970 -0.005344      33        1  1.256930
17    -0.061602  0.002205      34        1  1.231538
14    -0.049582  0.020377      35        1  1.227437
19     0.002952  0.014499      36        1  1.112788
25    -0.061602  0.002205      37        1  1.084519
20    -0.159773 -0.012565      38        1  0.993294
10    -0.078183  0.007888      39        1  0.983995
18    -0.061602  0.002205      40        1  0.906820
39    -0.061602  0.002205      41        1  0.813360
12    -0.061602  0.002205      42        1  0.785951
36    -0.061602  0.002205      43        1  0.765313
2     -0.061602  0.002205      44        1  0.723481
32    -0.061602  0.002205      45        1  0.598716
24    -0.061602  0.002205      46        1  0.403607, topic_info=            Term   Freq       Total Category  logprob  loglift
37   подмосковье  194.0  194.000000  Default     30.0     30.0
7       получать  154.0  154.000000  Default     29.0     29.0
5         задача  193.0  193.000000  Default     28.0     28.0
8       институт  142.0  142.000000  Default     27.0     27.0
324        образ   77.0   77.000000  Default     26.0     26.0
..           ...    ...         ...      ...      ...      ...
27           сми    0.0   49.390384  Topic46     -inf     -inf
28            ру    0.0   38.413836  Topic46     -inf     -inf
29          кадр    0.0   88.133814  Topic46     -inf     -inf
30       насилие    0.0   27.465739  Topic46     -inf     -inf
31       вырасти    0.0   75.451004  Topic46     -inf     -inf

[1539 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
564      16  0.984211      daily
521      21  1.011247   facebook
122       2  0.579403  instagram
122       4  0.429187  instagram
141       9  0.564977       news
...     ...       ...        ...
366       7  0.512821       язык
366       9  0.487180       язык
896      10  0.988849         ян
208       8  1.002302     ян кап
903      30  0.978533     январь

[831 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 27, 45, 46, 41, 9, 10, 42, 17, 14, 32, 24, 23, 43, 35, 4, 38, 1, 29, 31, 12, 5, 8,

Вывод: исходя из визуализации, результаты работы bigartm хорошие